# Chapter 8 Object References, Mutability and Recycling

## Variables are not Boxes
They are tags on objects. A variable is assigned to an object, not the other way around.

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
print(b)

[1, 2, 3, 4]


In [3]:
# Example 8-2. Variables are assigned to objects only after the objects are created
class Gizmo:
    def __init__(self):
        print('Gizmo id: {}'.format(id(self)))

x = Gizmo()
y = Gizmo() * 10

Gizmo id: 140438589093480
Gizmo id: 140438589093536


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

## Identity, Equality, and Aliases

* Identitiy: Two objects has the same id. It is checked by the `is` operator.
* Equality: Two objects has the same values. It is checked by the `==` operator.

In [6]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
print('lewis is charles:', lewis is charles)
print('id of charles: {}. id of lewis: {}'.format(id(charles), id(lewis)))
lewis['balance'] = 950
print(charles)

lewis is charles: True
id of charles: 140438345536568. id of lewis: 140438345536568
{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}


In [7]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
print('alex == charles', alex == charles)
print('alex is not charles', alex is not charles)

alex == charles True
alex is not charles True


### Choosing Between == and is

Generally, `==` is more frequently used than `is`. However, use `is` when you compare a variable to a singleton (extremely case `None`).
```
x is None
x is not None
```
`is` is faster than `==` because `is` cannot be overloaded.

### The Relative Immutability of Tuples
Tuples hold reference to objects. If the referenced items are mutable, they may change.

In [9]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
print('t1 == t2', t1 == t2)
print('id of t1:', id(t1[-1]))
t1[-1].append(99)
print(t1)
print('id of t1:', id(t1[-1]))
print('t1 == t2', t1 == t2)

t1 == t2 True
id of t1: 140438346638536
(1, 2, [30, 40, 99])
id of t1: 140438346638536
t1 == t2 False


## Copies Are Shallow by Default

Using the constructor `list()` or `[:]` produces a shallow copy. The outermost container is duplicated, but the copy is filled with references to the sam items held by the original container.

In [2]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)
print('l1:', l1)
print('l2:', l2)
l1[1] += [33, 22]  # for a mutable object, the operator += cahnges the list in place
l2[2] += (10, 11)  # create a new tuple and rebinds the variable l2[2]
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


Before the operation
```
l1 --->  | 3 | list | tuple |
l2 --->  | 3 | list | tuple |
list --> [66, 55, 44]
tuple --> (7, 8, 9)
```

After the operation
```
l1 --->  | 3 | list | tuple1 |
l2 --->  | 3 | list | tuple2 |
list --> [66, 44, 33, 22]
tuple1 --> (7, 8, 9)
tuple2 --> (7, 8, 9, 10, 11)
```

### Deep and Shallow Copies of Arbitrary Objects

* `copy.copy` shallow. A shallow copy constructs a new compound object and then (to the extent possible) inserts references into it to the objects found in the original.
* `copy.deepcopy` deep. A deep copy constructs a new compound object and then, recursively, inserts copies into it of the objects found in the original.

In [5]:
from typing import List

class Bus:
    def __init__(self, passengers : List[str] = None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(self, name: str):
        self.passengers.append(name)
    
    def drop(self, name: str):
        self.passengers.remove(name)

In [6]:
import copy

bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
print('ids:', id(bus1), id(bus2), id(bus3))
bus1.drop('Bill')
print('passengers on bus 2:', bus2.passengers)
print('passenrgers.ids:', id(bus1.passengers), id(bus2.passengers), id(bus3.passengers))
print('passengers on bus 3:', bus3.passengers)

ids: 139894218686304 139894218686192 139894218685744
passengers on bus 2: ['Alice', 'Claire', 'David']
passenrgers.ids: 139894223563848 139894223563848 139894217906376
passengers on bus 3: ['Alice', 'Bill', 'Claire', 'David']


`deepcopy` address cylcic references

In [8]:
a = [10, 20]
b = [a, 30]
a.append(b)
print(a)
c = copy.deepcopy(a)
print(c)

[10, 20, [[...], 30]]
[10, 20, [[...], 30]]


## Funciton Parameters as References

The only mode is call by sharing, meaning each formal parameter of the function gets a copy of each reference in the arguments.

The result of this schema is that a function may change any mutatble object passed as a parameter, but it cannot change the identity of those objects.

In [10]:
def f(a, b):
    a += b
    return a

In [11]:
x, y = 1, 2
f(x, y)
print(x, y)

1 2


In [12]:
a = [1, 2]
b = [3, 4]
f(a, b)
print(a, b)

[1, 2, 3, 4] [3, 4]


In [13]:
t = (10, 20)
u = (30, 40)
f(t, u)
print(t, u)

(10, 20) (30, 40)


### Mutable Types as Parameter Defaults: Bad Idea

You should avoid mutable objects as default values for parameters.

In [21]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""
    def __init__(self, passengers: List[str] = []):
        self.passengers = passengers
    
    def pick(self, name: str):
        self.passengers.append(name)
    
    def drop(self, name: str):
        self.passengers.remove(name)

In [22]:
bus1 = HauntedBus()
print('passengers in bus 1: ', bus1.passengers)
bus1.pick('Charlie')
print('passengers in bus 1: ', bus1.passengers)
bus2 = HauntedBus()
print('passengers in bus 2: ', bus2.passengers)

passengers in bus 1:  []
passengers in bus 1:  ['Charlie']
passengers in bus 2:  ['Charlie']


`bus1` and `bus2` end up sharing the same list (default). Inspect the inner structure:

In [26]:
HauntedBus.__init__.__defaults__

(['Charlie'],)

### Defensive Programming with Mutable Parameters

When you are coding a function that receives a mutable parameter, you should carefully consider whether the caller expects the argument passed to be changed.

In [34]:
class TwilightBus(Bus):
    def __init__(self, passegners=None):
        self.passengers = passegners if passegners is not None else []

In [35]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
print(basketball_team)

['Sue', 'Maya', 'Diana']


## del and Garbage Collection

The `del` statement deletes names, not objects. An object is garbage collected when the reference count become 0.

In [38]:
# End the an object's life
import weakref
s1 = {1, 2, 3}
s2 = s1

ender = weakref.finalize(s1, lambda : print('Gone with the wind...'))
print('alive: ', ender.alive)
del s1
print('alive: ', ender.alive)
s2 = 'spam'
print('alive: ', ender.alive)

alive:  True
alive:  True
Gone with the wind...
alive:  False


## Weak References

The presence of references is what keeps an object alive in memory. When the reference count of an object reaches zero, the garbage collector disposes of it.

Weak references to an object do not include its reference count. The object that is the target of a reference is called the referent. A weak reference does not prevent the referent from being garbage collected.

In [44]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)
print('wref', wref)

print('wref val:', wref())
a_set = {2, 3, 4}
print('wref val is None:', wref() is None)
print('wref val:', wref())

wref <weakref at 0x7f3ba8c1aa98; to 'set' at 0x7f3ba8c70128>
wref val: {0, 1}
wref val is None: True
wref val: None


### The WeakValueDictionary Skit

The value are weak references to objects. This is commonly used for caching.

In [45]:
class Cheese:
    def __init__(self, kind: str):
        self.kind = kind
    
    def __repr__(self):
        return 'Cheese({})'.format(self.kind)

In [46]:
import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]

for cheese in catalog:
    stock[cheese.kind] = cheese
    
print(sorted(stock.keys()))
del catalog
print(sorted(stock.keys()))
del cheese
print(sorted(stock.keys()))

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']
['Parmesan']
[]
